## Devoir Topas - Équipe protons
### PHY-7051 Physique des radiations en radiothérapie et en radiologie
Gabrielle Germain (111 266 117), Hanieh Sadat Jozi (111 XXX XXX), Philippe Dionne (111 XXX XXX)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate
from matplotlib.patches import Rectangle
from scipy.stats import linregress